In [1]:
import pika
import sys
pika.__version__

'1.1.0'

In [2]:
credentials = pika.PlainCredentials('myguest', 'myguestpwd')
exchangeName = 'direct_logs'
#URL='localhost'
URL='PROD-JOB-844fd7d2202ac4da.elb.us-east-2.amazonaws.com'
parameters =  pika.ConnectionParameters(URL, port=5672, credentials=credentials)
connection = pika.BlockingConnection(parameters)
channel = connection.channel()

channel.exchange_declare(exchange=exchangeName, exchange_type='direct')


<METHOD(['channel_number=1', 'frame_type=1', 'method=<Exchange.DeclareOk>'])>

In [3]:
# We will run this function when we receive a message from the server/client
def callback(ch, method, properties, body):
    print(" [x] %r:%r" % (method.routing_key, body))

In [ ]:
myqueue = channel.queue_declare('', exclusive=True)
#myqueue = channel.queue_declare('task_queue')
queue_name = myqueue.method.queue
print('queue_name:', queue_name) # The queue name is not important, only the routing matters in this demo!

# This worker only wants to lisiten to messages with 'Pittsburgh' or 'Washington' routing
for myrouting in ['Pittsburgh','Washington']:
    #bind the routing_keys to the channel
    channel.queue_bind(exchange='direct_logs', queue=queue_name, routing_key=myrouting)

print(' [*] Waiting for logs. To exit press CTRL+C')

#We will call the callback function, when we receive a message
channel.basic_consume(queue=queue_name, on_message_callback=callback, auto_ack=True)

channel.start_consuming()

queue_name: amq.gen-lo52vPGp-qVCUVvc9xI98g
 [*] Waiting for logs. To exit press CTRL+C
 [x] 'Washington':b'Message1_Washington'
 [x] 'Washington':b'Message3_Washington'
 [x] 'Pittsburgh':b'Message6_Pittsburgh'
 [x] 'Pittsburgh':b'Message7_Pittsburgh'
 [x] 'Washington':b'Message9_Washington'
 [x] 'Pittsburgh':b'Message2_Pittsburgh'
 [x] 'Washington':b'Message3_Washington'
 [x] 'Washington':b'Message4_Washington'
 [x] 'Pittsburgh':b'Message6_Pittsburgh'
 [x] 'Pittsburgh':b'Message7_Pittsburgh'
 [x] 'Washington':b'Message0_Washington'
 [x] 'Pittsburgh':b'Message1_Pittsburgh'
 [x] 'Washington':b'Message2_Washington'
 [x] 'Pittsburgh':b'Message4_Pittsburgh'
 [x] 'Washington':b'Message5_Washington'
 [x] 'Washington':b'Message9_Washington'
 [x] 'Pittsburgh':b'Message0_Pittsburgh'
 [x] 'Pittsburgh':b'Message1_Pittsburgh'
 [x] 'Washington':b'Message2_Washington'
 [x] 'Pittsburgh':b'Message4_Pittsburgh'
 [x] 'Pittsburgh':b'Message6_Pittsburgh'
 [x] 'Pittsburgh':b'Message7_Pittsburgh'
 [x] 'Pitts